In [1]:
import os
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import open3d as o3d

from tqdm import tqdm
print(o3d.__version__)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
0.16.0


### 1. Visualisasikan wajah 3D mesh original

In [2]:
def load_dataset(directory):
    datasets = []
    labels = []
    
    for folder in os.listdir(directory):
        if folder.startswith('._'):
                continue
        for file in os.listdir(directory + '/' + folder):
            if file.startswith('._'):
                continue
                
            pcd = o3d.io.read_triangle_mesh(directory + '/' + folder + '/' + file)
            pcd.compute_vertex_normals()
            pcd.compute_triangle_normals()
            pcd.paint_uniform_color([0.5, 0.5, 0.5])
            pcd.normalize_normals()
            
            datasets.append(pcd)
            labels.append(folder)

    return datasets, labels

In [3]:
directory = 'FaceTalk_170725_00137_TA'

dataset, labels = load_dataset(directory)

In [4]:
print("Total data:", len(dataset))

Total data: 1183


In [5]:
viz_data = []

random.seed(42)
index = random.sample(range(1, 1183), 10)
print(index)

for i in index:
    viz_data.append(dataset[i])

[229, 52, 564, 502, 458, 286, 210, 1117, 179, 865]


In [6]:
origin_x = 0
origin_y = 0
origin_z = 0

#translate y to show 5 faces per row
for i in range(len(viz_data)):
    viz_data[i].translate([origin_x, origin_y, origin_z],False)
    origin_x += 0.3
    if i == 4:
        origin_x = 0
        origin_y += 0.4

In [7]:
o3d.visualization.draw_geometries(viz_data,window_name="Show 10 Faces", width=1024, height=768)

### 2. Latih model wajah 3D menggunakan PCA
- Pisahkan dataset asal menjadi 90% data training dan 10% data test. 
- Ukurlah performa model dengan membandingkan wajah asli dan wajah rekonstruksi baik pada data training maupun data test, secara kuantitatif dengan mean squared error

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
data_train, data_test = train_test_split(dataset, test_size=0.1, random_state=42)

In [10]:
print('Total train:', len(data_train))
print('Total test:', len(data_test))

Total train: 1064
Total test: 119


In [11]:
def get_vertices(dataset):
    vertices_arr = []
    for data in dataset:
        vertices = np.asarray(data.vertices)
        vertices_flat = vertices.reshape([1, -1])[0]
        vertices_arr.append(vertices_flat)
        
    return vertices_arr

In [12]:
vertices_train = get_vertices(data_train)
vertices_test = get_vertices(data_test)

In [13]:
vertices_train[:2]

[array([ 0.06624033, -0.00799667, -0.05483579, ..., -0.03353754,
         0.02016076,  0.01279395]),
 array([ 0.06601074, -0.00843078, -0.05426706, ..., -0.0329448 ,
         0.02120767,  0.01200421])]

In [14]:
vertices_train_df = pd.DataFrame(vertices_train)
vertices_test_df = pd.DataFrame(vertices_test)

vertices_train_df[:5]

,0,1,2,3,4,5,6,7,8,9,...,15059,15060,15061,15062,15063,15064,15065,15066,15067,15068
0,0.066240,-0.007997,-0.054836,0.069736,-0.009021,-0.053379,0.070293,-0.007498,-0.052928,0.066799,...,0.016228,-0.037964,0.019277,0.014669,-0.035842,0.019701,0.013508,-0.033538,0.020161,0.012794
1,0.066011,-0.008431,-0.054267,0.069492,-0.009473,-0.052801,0.070047,-0.007952,-0.052379,0.066559,...,0.015424,-0.037355,0.020318,0.013866,-0.035241,0.020745,0.012712,-0.032945,0.021208,0.012004
2,0.366231,0.391793,-0.054023,0.369716,0.390781,-0.052545,0.370283,0.392307,-0.052102,0.366801,...,0.016785,0.262194,0.419459,0.015229,0.264314,0.419883,0.014073,0.266616,0.420343,0.013362
3,0.066029,-0.008801,-0.054413,0.069506,-0.009808,-0.052929,0.070057,-0.008265,-0.052517,0.066573,...,0.015062,-0.037079,0.020940,0.013508,-0.034968,0.021366,0.012356,-0.032675,0.021829,0.011648
4,0.066157,-0.007826,-0.054637,0.069646,-0.008854,-0.053164,0.070205,-0.007330,-0.052720,0.066717,...,0.015982,-0.037691,0.019381,0.014421,-0.035570,0.019804,0.013262,-0.033267,0.020264,0.012550


In [15]:
print(vertices_train_df.shape)

(1064, 15069)


In [16]:
from sklearn.decomposition import PCA

In [17]:
pca_train = PCA(n_components=100)
pca_train.fit(vertices_train_df)
train_components = pca_train.transform(vertices_train_df)

pca_test = PCA(n_components=100)
pca_test.fit(vertices_test_df)
test_components = pca_test.transform(vertices_test_df)

In [18]:
train_components

array([[-4.07706359e-01,  1.13952340e-02, -4.67818177e-02, ...,
         1.69399560e-04, -1.35058894e-04,  2.39942623e-04],
       [-3.73386784e-01, -9.53505581e-03,  5.27312717e-02, ...,
        -1.21265281e-03, -3.46872203e-04,  3.37963325e-04],
       [ 2.68460433e+01,  2.26703520e+01, -3.86043836e-03, ...,
        -6.94598664e-06, -3.35531407e-04,  1.61896270e-03],
       ...,
       [-4.17398377e-01,  7.74092452e-03, -1.48678364e-01, ...,
         1.81267694e-03,  3.39361683e-04, -2.51800166e-04],
       [-4.53640331e-01,  2.16486404e-02, -2.68850152e-02, ...,
         3.14781289e-04, -2.57024585e-04, -2.18974043e-03],
       [-4.31270379e-01,  1.93779238e-02, -1.55021073e-01, ...,
        -2.19670602e-03, -8.96403792e-05, -2.97487639e-03]])

In [19]:
train_components_df = pd.DataFrame(train_components)
test_components_df = pd.DataFrame(test_components)

train_components_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.407706,0.011395,-0.046782,-0.070023,0.004977,-0.046691,0.010405,-0.003655,0.011026,-0.009550,...,0.000429,0.001068,0.000140,-0.000432,-0.002112,-0.000200,-0.000232,0.000169,-0.000135,0.000240
1,-0.373387,-0.009535,0.052731,-0.090786,-0.148792,-0.020914,0.046293,-0.011828,0.023533,0.049814,...,-0.000323,0.001598,0.000013,-0.001731,-0.001286,0.000964,-0.001444,-0.001213,-0.000347,0.000338
2,26.846043,22.670352,-0.003860,0.095406,-0.075503,-0.061940,0.009453,-0.043809,0.014728,-0.019661,...,-0.000282,-0.002098,-0.001493,-0.000772,0.000089,-0.000098,0.003289,-0.000007,-0.000336,0.001619
3,-0.437951,-0.056042,0.223695,-0.085213,0.022171,0.102730,0.081311,0.009694,0.003163,-0.000526,...,-0.001221,-0.002069,-0.000613,0.001208,-0.000542,0.000176,-0.000635,-0.001177,0.001893,0.000614
4,-0.405258,0.012369,-0.040759,-0.103635,-0.010595,-0.028124,-0.003967,-0.019856,0.001434,-0.004998,...,0.000549,-0.000686,0.000250,-0.000412,-0.001251,0.000292,0.000098,-0.000684,-0.000661,0.000432


In [20]:
# Inverse transform
inversed_vertices_train = pca_train.inverse_transform(train_components_df)
inversed_vertices_test = pca_test.inverse_transform(test_components_df)

In [21]:
inversed_vertices_train[:5]

array([[ 0.06624733, -0.00799803, -0.05482771, ..., -0.03354575,
         0.02015122,  0.01280337],
       [ 0.06599063, -0.00844932, -0.05423628, ..., -0.03294841,
         0.02122348,  0.01200459],
       [ 0.36622498,  0.39180385, -0.05401168, ...,  0.26662132,
         0.42033499,  0.01337451],
       [ 0.06604318, -0.00880676, -0.05442896, ..., -0.03268252,
         0.02182637,  0.01164785],
       [ 0.06617435, -0.00781912, -0.05465769, ..., -0.03327488,
         0.0202668 ,  0.01254078]])

In [22]:
len(inversed_vertices_train)

1064

In [23]:
from sklearn.metrics import mean_squared_error

In [24]:
inversed_vertices_train_df = pd.DataFrame(inversed_vertices_train)
inversed_vertices_test_df = pd.DataFrame(inversed_vertices_test)

print("MSE Train:", mean_squared_error(vertices_train_df, inversed_vertices_train_df))
print("MSE Test:", mean_squared_error(vertices_test_df, inversed_vertices_test_df))

MSE Train: 8.41791381304448e-09
MSE Test: 6.375274903753317e-10


### 3. Visualisasikan hasil rekonstruksi wajah 3D mesh
Visualisasikan hasil rekonstruksi wajah 3D mesh dari beberapa sampel pada data test, minimal 10 wajah

In [25]:
def convert_to_vertices(inversed):
    vertices_arr = []
    for i in range(len(inversed)):
        new_vertices = np.reshape(inversed[i], (-1, 3))
        vertices_arr.append(new_vertices)
    
    return vertices_arr

In [26]:
inversed_vertices_train = convert_to_vertices(inversed_vertices_train)
inversed_vertices_test = convert_to_vertices(inversed_vertices_test)

In [27]:
def convert_to_meshes(inversed_vertices, dataset):
    meshes = []
    for i in range(len(inversed_vertices)):
        new_tri = np.asarray(dataset[i].triangles)
        
        new_mesh = o3d.geometry.TriangleMesh(
            o3d.cpu.pybind.utility.Vector3dVector(inversed_vertices[i]), 
            o3d.cpu.pybind.utility.Vector3iVector(new_tri))
        
        new_mesh.compute_vertex_normals()
        new_mesh.compute_triangle_normals()
        new_mesh.paint_uniform_color([0.5, 0.5, 0.5])
        new_mesh.normalize_normals()
        meshes.append(new_mesh)
            
    return meshes

In [28]:
results_train = convert_to_meshes(inversed_vertices_train, data_train)
results_test = convert_to_meshes(inversed_vertices_test, data_test)

In [29]:
viz_data = []

random.seed(42)
index = random.sample(range(1, 119), 10)
print(index)

for i in index:
    viz_data.append(results_test[i])

[82, 15, 4, 95, 36, 32, 29, 18, 14, 87]


In [30]:
origin_x = 0
origin_y = 0
origin_z = 0

# Show pca 3d model
for i in range(len(viz_data)):
    viz_data[i].translate([origin_x, origin_y, origin_z],False)
    origin_x += 0.3
    if i in [4, 9, 14]:
        origin_x = 0
        origin_y += 0.4

In [31]:
o3d.visualization.draw_geometries(viz_data,window_name="Show 10 Faces (PCA)", width=1024, height=768)

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


### 4. Visualisasikan dekodifikasi wajah 3D mesh 
Visualisasikan dekodifikasi wajah 3D mesh dengan mengubah-ubah nilai salah satu kode atau elemen pada variabel latent.

In [32]:
train_components_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.407706,0.011395,-0.046782,-0.070023,0.004977,-0.046691,0.010405,-0.003655,0.011026,-0.009550,...,0.000429,0.001068,0.000140,-0.000432,-0.002112,-0.000200,-0.000232,0.000169,-0.000135,0.000240
1,-0.373387,-0.009535,0.052731,-0.090786,-0.148792,-0.020914,0.046293,-0.011828,0.023533,0.049814,...,-0.000323,0.001598,0.000013,-0.001731,-0.001286,0.000964,-0.001444,-0.001213,-0.000347,0.000338
2,26.846043,22.670352,-0.003860,0.095406,-0.075503,-0.061940,0.009453,-0.043809,0.014728,-0.019661,...,-0.000282,-0.002098,-0.001493,-0.000772,0.000089,-0.000098,0.003289,-0.000007,-0.000336,0.001619
3,-0.437951,-0.056042,0.223695,-0.085213,0.022171,0.102730,0.081311,0.009694,0.003163,-0.000526,...,-0.001221,-0.002069,-0.000613,0.001208,-0.000542,0.000176,-0.000635,-0.001177,0.001893,0.000614
4,-0.405258,0.012369,-0.040759,-0.103635,-0.010595,-0.028124,-0.003967,-0.019856,0.001434,-0.004998,...,0.000549,-0.000686,0.000250,-0.000412,-0.001251,0.000292,0.000098,-0.000684,-0.000661,0.000432


In [33]:
train_components_df[5] = 0.2
test_components_df[5] = 0.2

In [35]:
test_components_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.304185,0.296476,0.116393,-0.020638,-0.026603,0.2,-0.022372,-0.011563,0.012371,0.002216,...,0.000191,-0.000799,0.000169,0.000373,-0.000430,0.000050,-0.000135,0.000062,0.000160,-0.000621
1,-0.226014,-0.117398,-0.128399,0.132194,0.062564,0.2,0.014799,-0.008130,0.001470,0.006010,...,-0.000797,-0.001977,0.000814,-0.001539,-0.000409,-0.001960,-0.000645,-0.001375,-0.001104,0.000878
2,-0.241554,0.069496,-0.069971,0.011131,-0.021923,0.2,-0.015185,-0.018618,-0.003265,-0.017363,...,0.002819,-0.001035,0.001388,-0.000291,-0.002368,-0.000305,-0.000869,0.000877,0.004407,-0.000712
3,-0.235915,-0.063871,0.161037,0.003271,-0.048055,0.2,-0.024539,-0.017103,0.000062,0.010350,...,-0.000607,0.000427,-0.000280,0.000421,0.000841,-0.000644,0.000035,-0.000088,0.000096,0.000281
4,-0.211970,-0.045098,-0.086172,0.012568,-0.030065,0.2,-0.036575,0.013960,0.007604,-0.008844,...,0.000724,0.000312,-0.000608,0.001078,-0.001317,-0.000226,0.000522,0.001456,-0.000284,0.000976
5,-0.209169,-0.097251,-0.049430,-0.019345,-0.017877,0.2,-0.023497,0.009405,0.014754,-0.015146,...,-0.000604,-0.000686,0.000143,0.000515,0.001217,0.000362,-0.000377,0.000369,-0.000307,-0.000830
6,-0.229410,-0.106802,-0.115802,0.172344,0.064978,0.2,0.014131,-0.008412,0.004113,0.006040,...,0.001351,0.002297,-0.000753,0.001944,0.000181,0.002055,0.000541,0.001427,0.001085,-0.000965
7,-0.316145,0.403390,-0.011342,-0.017930,0.016509,0.2,0.032439,0.000843,-0.034472,-0.001938,...,0.001587,-0.001823,-0.000324,0.000066,0.000834,-0.001120,0.002073,-0.002673,-0.000493,0.000591
8,-0.216147,-0.089987,-0.041672,-0.011741,-0.011171,0.2,-0.019350,0.012623,0.019304,0.001509,...,-0.000190,0.000268,-0.000798,0.000469,-0.000717,0.000679,0.001099,0.001420,-0.001009,0.001173
9,-0.214211,-0.066095,-0.056034,-0.005977,-0.060585,0.2,0.060187,0.001526,0.004428,0.009842,...,-0.001053,0.001629,-0.001073,-0.002188,0.000057,0.001109,0.000499,0.000448,0.000435,0.000205


In [ ]:
# Mulut
# train_components_df[0] = 0.2
# test_components_df[0] = 0.2

# Alis
# train_components_df[5] = 0.2
# test_components_df[5] = 0.2

# Pipi
# train_components_df[8] = 0.1
# test_components_df[8] = 0.1

In [36]:
# Inverse transform
inversed_vertices_train = pca_train.inverse_transform(train_components_df)
inversed_vertices_test = pca_test.inverse_transform(test_components_df)

In [37]:
inversed_vertices_train_df = pd.DataFrame(inversed_vertices_train)
inversed_vertices_test_df = pd.DataFrame(inversed_vertices_test)

# print(mean_squared_error(vertices_train_df, inversed_vertices_train_df))
# print(mean_squared_error(vertices_test_df, inversed_vertices_test_df))

In [38]:
inversed_vertices_train = convert_to_vertices(inversed_vertices_train)
inversed_vertices_test = convert_to_vertices(inversed_vertices_test)

In [39]:
results_train = convert_to_meshes(inversed_vertices_train, data_train)
results_test = convert_to_meshes(inversed_vertices_test, data_test)

In [40]:
viz_data = []

random.seed(42)
index = random.sample(range(1, 119), 10)
print(index)

for i in index:
    viz_data.append(results_test[i])

[82, 15, 4, 95, 36, 32, 29, 18, 14, 87]


In [41]:
origin_x = 0
origin_y = 0
origin_z = 0

#translate origin to show 5 faces per row
for i in range(len(viz_data)):
    viz_data[i].translate([origin_x, origin_y, origin_z],False)
    origin_x += 0.3
    if i in [4, 9, 14]:
        origin_x = 0
        origin_y += 0.4

In [42]:
# Show pca 3d model with modified latent
o3d.visualization.draw_geometries(viz_data,window_name="Show 10 Faces (PCA - Latent Modified)", width=1024, height=768)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


### 5. Visualisasikan 20 principal components dengan eigenvalues tertinggi 

In [43]:
eigenvalues_train = pca_train.explained_variance_
eigenvalues_test = pca_test.explained_variance_

eigenvalues_train[:20]

array([2.67007474e+01, 1.67757558e+00, 3.37208588e-02, 2.02075024e-02,
       5.06421609e-03, 2.20436663e-03, 1.47642034e-03, 7.90686351e-04,
       5.03488131e-04, 4.69131645e-04, 3.11621271e-04, 2.43905381e-04,
       2.02006677e-04, 1.28602294e-04, 1.10036802e-04, 9.06904595e-05,
       7.45165379e-05, 6.41310780e-05, 5.86637603e-05, 4.89917827e-05])

In [46]:
pd.DataFrame(eigenvalues_train[:20].reshape([1,-1]))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,26.700747,1.677576,0.033721,0.020208,0.005064,0.002204,0.001476,0.000791,0.000503,0.000469,0.000312,0.000244,0.000202,0.000129,0.00011,0.000091,0.000075,0.000064,0.000059,0.000049


### 6. Autoencoder

#### Latih model wajah 3D secara unsupervised dengan Autoencoder
Ukurlah performa model dengan membandingkan wajah asli dan wajah rekonstruksi baik pada data training maupun data test, secara kuantitatif dengan mean squared error.

In [47]:
import torch
import torchvision.transforms as T

In [48]:
EPOCHS = 20
BATCH_SIZE = 32
LEARNING_RATE = 1e-3

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [49]:
class AE(torch.nn.Module):
    def __init__(self, d_in=15069):
        super().__init__()
         
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(d_in, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 100),
        )
        
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(100, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, d_in),
            torch.nn.Tanh()
        )
 
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
    def encode(self, x):
        return self.encoder(x)
    
    def decode(self, encoded):
        return self.decoder(encoded)

In [50]:
# Model Initialization
model = AE(d_in=15069).to(DEVICE)
print(f"Using {DEVICE} device")
print(model)
 
# Validation using MSE Loss function
loss_function = torch.nn.MSELoss()
 
# Using an Adam Optimizer with lr = 0.1
optimizer = torch.optim.Adam(model.parameters(),
                             lr = LEARNING_RATE)

Using cuda device
AE(
  (encoder): Sequential(
    (0): Linear(in_features=15069, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=100, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=100, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): ReLU()
    (6): Linear(in_features=1024, out_features=15069, bias=True)
    (7): Tanh()
  )
)


In [51]:
vertices_train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,15059,15060,15061,15062,15063,15064,15065,15066,15067,15068
0,0.066240,-0.007997,-0.054836,0.069736,-0.009021,-0.053379,0.070293,-0.007498,-0.052928,0.066799,...,0.016228,-0.037964,0.019277,0.014669,-0.035842,0.019701,0.013508,-0.033538,0.020161,0.012794
1,0.066011,-0.008431,-0.054267,0.069492,-0.009473,-0.052801,0.070047,-0.007952,-0.052379,0.066559,...,0.015424,-0.037355,0.020318,0.013866,-0.035241,0.020745,0.012712,-0.032945,0.021208,0.012004
2,0.366231,0.391793,-0.054023,0.369716,0.390781,-0.052545,0.370283,0.392307,-0.052102,0.366801,...,0.016785,0.262194,0.419459,0.015229,0.264314,0.419883,0.014073,0.266616,0.420343,0.013362
3,0.066029,-0.008801,-0.054413,0.069506,-0.009808,-0.052929,0.070057,-0.008265,-0.052517,0.066573,...,0.015062,-0.037079,0.020940,0.013508,-0.034968,0.021366,0.012356,-0.032675,0.021829,0.011648
4,0.066157,-0.007826,-0.054637,0.069646,-0.008854,-0.053164,0.070205,-0.007330,-0.052720,0.066717,...,0.015982,-0.037691,0.019381,0.014421,-0.035570,0.019804,0.013262,-0.033267,0.020264,0.012550


In [52]:
def convert_to_tensors(vertices_df):
    # Convert to numpy arrays
    arr = vertices_df.to_numpy().astype(np.float32)
    
    # Convert to tensors
    tensr = torch.from_numpy(arr)
    
    return tensr

In [53]:
tensor_train = convert_to_tensors(vertices_train_df)
tensor_test = convert_to_tensors(vertices_test_df)

tensor_train

tensor([[ 0.0662, -0.0080, -0.0548,  ..., -0.0335,  0.0202,  0.0128],
        [ 0.0660, -0.0084, -0.0543,  ..., -0.0329,  0.0212,  0.0120],
        [ 0.3662,  0.3918, -0.0540,  ...,  0.2666,  0.4203,  0.0134],
        ...,
        [ 0.0657, -0.0086, -0.0559,  ..., -0.0333,  0.0199,  0.0129],
        [ 0.0661, -0.0066, -0.0555,  ..., -0.0330,  0.0198,  0.0124],
        [ 0.0658, -0.0079, -0.0561,  ..., -0.0333,  0.0197,  0.0128]])

In [54]:
# Convert to dataloaders
train_loader = torch.utils.data.DataLoader(dataset=tensor_train,                                     
                                           batch_size=BATCH_SIZE)
test_loader = torch.utils.data.DataLoader(dataset=tensor_test,                                     
                                           batch_size=BATCH_SIZE)

In [55]:
# Train
def train_model(model, train_loader):
    for epoch in range(EPOCHS):
        loss = 0
        size = len(train_loader.dataset)
        recons = []
        for batch, X in enumerate(train_loader):
            X = X.to(DEVICE)

            # Compute prediction error
            reconstructed = model(X)
            train_loss = loss_function(reconstructed, X)

            # Backpropagation
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

            loss += train_loss.item()
            recons.append(reconstructed)

        # compute the epoch training loss
        loss = loss / len(train_loader)

        # display the epoch training loss
        print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, EPOCHS, loss))
    
    return model

In [56]:
autoencoder = train_model(model, train_loader)

epoch : 1/20, loss = 0.002741
epoch : 2/20, loss = 0.001171
epoch : 3/20, loss = 0.000771
epoch : 4/20, loss = 0.000981
epoch : 5/20, loss = 0.000388
epoch : 6/20, loss = 0.000269
epoch : 7/20, loss = 0.000341
epoch : 8/20, loss = 0.000357
epoch : 9/20, loss = 0.000402
epoch : 10/20, loss = 0.000342
epoch : 11/20, loss = 0.000504
epoch : 12/20, loss = 0.000675
epoch : 13/20, loss = 0.000597
epoch : 14/20, loss = 0.000662
epoch : 15/20, loss = 0.000323
epoch : 16/20, loss = 0.000240
epoch : 17/20, loss = 0.000213
epoch : 18/20, loss = 0.000170
epoch : 19/20, loss = 0.000178
epoch : 20/20, loss = 0.000168


In [57]:
# Encode
def encode_data(loader, model):
    encoded = []
    for batch, X in enumerate(loader):
        
        X = X.to(DEVICE)
        enc = model.encode(X)
        encoded.append(enc)
        
    return encoded

In [58]:
encoded_train = encode_data(train_loader, autoencoder)
encoded_test = encode_data(test_loader, autoencoder)

In [59]:
# Get latent variables
def get_latents(encoded):
    latents = []
    for item in encoded:
        for i in item:
            t = i.cpu()
            t = t.detach().numpy()
            latents.append(t)
    return pd.DataFrame(latents)

In [60]:
latent_train_df = get_latents(encoded_train)
latent_test_df = get_latents(encoded_test)

latent_train_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.113239,0.024670,-0.092579,0.091130,0.151312,-0.061137,-0.054284,-0.137557,-0.009897,-0.102400,...,0.000743,0.102453,0.054127,-0.016629,0.358468,0.042317,0.226967,-0.053832,0.262848,0.265659
1,0.113133,0.024725,-0.092541,0.091151,0.151164,-0.061073,-0.054209,-0.137468,-0.009896,-0.102344,...,0.000773,0.102367,0.054066,-0.016634,0.358286,0.042277,0.226848,-0.053852,0.262687,0.265520
2,-0.136246,0.109647,0.098362,-0.131915,-0.024014,0.022369,0.194874,-0.059654,-0.133763,0.166692,...,0.045063,-0.029465,0.081855,-0.028852,0.004797,-0.002310,0.202494,-0.006225,0.035516,0.091467
3,0.115665,0.024050,-0.093374,0.091805,0.154522,-0.062506,-0.054648,-0.140019,-0.008864,-0.103665,...,0.000550,0.103947,0.054641,-0.016168,0.363770,0.043973,0.229518,-0.054372,0.266875,0.267892
4,0.113522,0.024593,-0.092671,0.091212,0.151686,-0.061308,-0.054333,-0.137862,-0.009772,-0.102544,...,0.000724,0.102635,0.054193,-0.016577,0.359110,0.042512,0.227281,-0.053900,0.263333,0.265926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059,0.115570,0.024076,-0.093346,0.091793,0.154401,-0.062478,-0.054636,-0.139967,-0.008883,-0.103610,...,0.000563,0.103908,0.054626,-0.016186,0.363623,0.043918,0.229456,-0.054376,0.266755,0.267814
1060,0.113522,0.024595,-0.092668,0.091212,0.151682,-0.061299,-0.054327,-0.137851,-0.009778,-0.102543,...,0.000720,0.102632,0.054190,-0.016575,0.359093,0.042510,0.227269,-0.053896,0.263321,0.265922
1061,0.113565,0.024566,-0.092658,0.091241,0.151703,-0.061331,-0.054335,-0.137929,-0.009770,-0.102533,...,0.000714,0.102683,0.054221,-0.016572,0.359222,0.042541,0.227324,-0.053905,0.263398,0.265964
1062,0.114401,0.024337,-0.092969,0.091393,0.152864,-0.061811,-0.054536,-0.138730,-0.009444,-0.103009,...,0.000632,0.103196,0.054422,-0.016432,0.360972,0.043076,0.228216,-0.054048,0.264776,0.266781


In [61]:
# Decode
def decode_data(latent, model):
    decoded = []
    for i in latent.index:
        X = latent.iloc[i]
        X = np.asarray(X).astype(np.float32)
        X = torch.from_numpy(X)
        X = X.to(DEVICE)
        dec = model.decode(X)
        decoded.append(dec.cpu())
    
    res = []
    for i in decoded:
        t = i.cpu()
        t = t.detach().numpy()
        res.append(t)
    
    return pd.DataFrame(res)

In [62]:
decoded_train_df = decode_data(latent_train_df, autoencoder)
decoded_test_df = decode_data(latent_test_df, autoencoder)

In [63]:
print("MSE Train:", mean_squared_error(vertices_train_df, decoded_train_df))
print("MSE Test:", mean_squared_error(vertices_test_df, decoded_test_df))

MSE Train: 0.000152322758943767
MSE Test: 0.000433487933580046


#### Visualisasikan hasil rekonstruksi wajah 3D mesh dari beberapa sampel pada data test

In [64]:
np.asarray(decoded_train_df)

array([[ 0.06617108, -0.00865226, -0.05491744, ..., -0.03310123,
         0.01956121,  0.01267362],
       [ 0.06617914, -0.00865099, -0.05491643, ..., -0.03309408,
         0.01956259,  0.01267401],
       [ 0.36399946,  0.06980088, -0.05144421, ...,  0.25584036,
         0.08195317,  0.00158799],
       ...,
       [ 0.06614297, -0.00865686, -0.05492152, ..., -0.03312753,
         0.01955921,  0.012676  ],
       [ 0.06607042, -0.00866996, -0.05493412, ..., -0.03319607,
         0.01955305,  0.01268114],
       [ 0.06613691, -0.00865796, -0.05492246, ..., -0.03313309,
         0.01955848,  0.0126761 ]], dtype=float32)

In [65]:
decoded_train = convert_to_vertices(np.asarray(decoded_train_df))
decoded_test = convert_to_vertices(np.asarray(decoded_test_df))

In [66]:
results_train = convert_to_meshes(decoded_train, data_train)
results_test = convert_to_meshes(decoded_test, data_test)

In [70]:
viz_data = []

random.seed(42)
index = random.sample(range(1, 119), 10)
print(index)

for i in index:
    viz_data.append(results_test[i])

[82, 15, 4, 95, 36, 32, 29, 18, 14, 87]


In [71]:
origin_x = 0
origin_y = 0
origin_z = 0

#translate origin to show 5 faces per row
for i in range(len(viz_data)):
    viz_data[i].translate([origin_x, origin_y, origin_z],False)
    origin_x += 0.3
    if i in [4, 9, 14]:
        origin_x = 0
        origin_y += 0.4

In [72]:
# Show autoencoder 3d model
o3d.visualization.draw_geometries(viz_data,window_name="Show 10 Faces (Autoencoder)", width=1024, height=768)

#### Visualisasikan wajah 3D mesh dengan mengubah nilai salah satu kode atau elemen pada variabel latent 

In [74]:
# Modify latent variables
latent_train_df[0] = 0.5
latent_test_df[0] = 0.5

In [75]:
latent_test_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.5,0.024107,-0.093342,0.091760,0.154341,-0.062420,-0.054608,-0.139850,-0.008912,-0.103603,...,0.000564,0.103835,0.054589,-0.016182,0.363419,0.043870,0.229344,-0.054350,0.266618,0.267738
1,0.5,0.024743,-0.092463,0.091118,0.150922,-0.060987,-0.054191,-0.137335,-0.009989,-0.102229,...,0.000773,0.102301,0.054060,-0.016665,0.357962,0.042165,0.226688,-0.053805,0.262426,0.265388
2,0.5,0.024327,-0.093014,0.091516,0.153075,-0.061907,-0.054484,-0.138946,-0.009315,-0.103088,...,0.000642,0.103291,0.054415,-0.016376,0.361433,0.043236,0.228398,-0.054146,0.265091,0.266894
3,0.5,0.024708,-0.092535,0.091111,0.151142,-0.061052,-0.054230,-0.137426,-0.009949,-0.102333,...,0.000747,0.102375,0.054089,-0.016638,0.358190,0.042240,0.226812,-0.053807,0.262627,0.265530
4,0.5,0.024663,-0.092587,0.091137,0.151342,-0.061150,-0.054286,-0.137579,-0.009885,-0.102414,...,0.000744,0.102463,0.054130,-0.016624,0.358517,0.042332,0.226991,-0.053838,0.262886,0.265679


In [76]:
decoded_train_df = decode_data(latent_train_df, autoencoder)
decoded_test_df = decode_data(latent_test_df, autoencoder)

In [78]:
decoded_train = convert_to_vertices(np.asarray(decoded_train_df))
decoded_test = convert_to_vertices(np.asarray(decoded_test_df))

In [79]:
results_train = convert_to_meshes(decoded_train, data_train)
results_test = convert_to_meshes(decoded_test, data_test)

In [81]:
viz_data = []

random.seed(42)
index = random.sample(range(1, 119), 10)
print(index)

for i in index:
    viz_data.append(results_test[i])

[82, 15, 4, 95, 36, 32, 29, 18, 14, 87]


In [82]:
origin_x = 0
origin_y = 0
origin_z = 0

#translate origin to show 5 faces per row
for i in range(len(viz_data)):
    viz_data[i].translate([origin_x, origin_y, origin_z],False)
    origin_x += 0.3
    if i in [4, 9, 14]:
        origin_x = 0
        origin_y += 0.4

In [83]:
# Show autoencoder 3d model with modified latent
o3d.visualization.draw_geometries(viz_data,window_name="Show 20 Faces (AE - Latent Modified)", width=1024, height=768)

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
